In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Send a GET request to the Wikipedia page
url = 'https://en.wikipedia.org/wiki/Glory_(kickboxing)'
response = requests.get(url)

# Create a BeautifulSoup object with the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the events table on the page
events_table = soup.find('table', class_='wikitable')


In [3]:
def get_events_df(table):
    headers = [_.text.strip() for _ in table.find_all('th')]
    
    rows = [
        [td.text.strip() for td in tr.find_all('td')]
        for tr in table.find_all('tr')[1:]
        ]
    
    df = pd.DataFrame(rows, columns=headers)

    return df

In [4]:
df = get_events_df(events_table)

In [5]:
df

,#,Event,Date,Venue,Location,Attendance
0,102,Glory: Collision 5,"June 17, 2023",Rotterdam Ahoy,"Rotterdam, Netherlands",
1,101,Glory 86,"May 27, 2023",Grugahalle,"Essen, Germany",
2,100,Glory 85,"April 29, 2023",Rotterdam Ahoy,"Rotterdam, Netherlands",
3,99,Glory 84,"March 11, 2023",Topsportcentrum,"Rotterdam, The Netherlands",
4,98,Glory 83,"February 11, 2023",Grugahalle,"Essen, Germany",
...,...,...,...,...,...,...
97,5,Road to Glory USA 95 kg Tournament,"February 2, 2013",Hard Rock Hotel and Casino,"Tulsa, Oklahoma, USA",
98,4,Dream 18/Glory 4: Tokyo,"December 31, 2012",Saitama Super Arena,"Saitama, Japan","15,000"
99,3,Glory 3: Rome,"November 3, 2012",PalaLottomatica,"Rome, Italy","6,450"
100,2,Glory 2: Brussels,"October 6, 2012",Forest National,"Brussels, Belgium","5,250"
